# Module 8: Similarity and Embeddings

**Goal:** Turn text into numbers that capture meaning

**Time:** ~20 minutes

**What you'll do:**
1. Create TF-IDF embeddings from support tickets
2. Find similar tickets using cosine similarity
3. Build a simple semantic search
4. Understand neural embeddings (preview)

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Load support tickets
tickets = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/data/streamcart_tickets.csv')

print(f"Loaded {len(tickets):,} support tickets")
tickets.head()

In [ ]:
# Look at some examples
print("=== Sample Tickets ===")
for i, row in tickets.sample(5, random_state=42).iterrows():
    print(f"[{row['category']}] {row['ticket_text']}")
    print()

---

## Part 1: TF-IDF Embeddings

Turn text into vectors where each dimension = importance of a word.

In [ ]:
# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(
    max_features=500,      # Keep top 500 words
    stop_words='english',  # Remove common words
    ngram_range=(1, 2)     # Include 2-word phrases
)

# Transform tickets to vectors
ticket_vectors = tfidf.fit_transform(tickets['ticket_text'])

print(f"Original: {len(tickets)} tickets with variable-length text")
print(f"After TF-IDF: {ticket_vectors.shape[0]} tickets × {ticket_vectors.shape[1]} dimensions")

In [ ]:
# Look at what TF-IDF learned
feature_names = tfidf.get_feature_names_out()

# Top words
word_importance = np.array(ticket_vectors.sum(axis=0)).flatten()
top_indices = word_importance.argsort()[-20:]

print("=== Most Important Terms ===")
for idx in reversed(top_indices):
    print(f"  {feature_names[idx]}: {word_importance[idx]:.1f}")

---

## Part 2: Finding Similar Tickets

Cosine similarity: how much do two vectors point in the same direction?

In [ ]:
# Pick a query ticket
query_idx = 0
query_text = tickets.iloc[query_idx]['ticket_text']
query_vector = ticket_vectors[query_idx]

print(f"Query ticket: '{query_text}'")
print(f"Category: {tickets.iloc[query_idx]['category']}")

In [ ]:
# TODO: Find 5 most similar tickets
#
# Steps:
# 1. Calculate cosine similarity between query and all tickets
# 2. Sort by similarity (descending)
# 3. Return top 5 (excluding the query itself)

similarities = None  # Replace with your code

# Uncomment when ready:
# similarities = cosine_similarity(query_vector, ticket_vectors)[0]

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

assert similarities is not None, "Calculate similarities first!"
assert len(similarities) == len(tickets), "Should have one score per ticket"

# Get top 5 similar (excluding query)
top_indices = similarities.argsort()[::-1][1:6]

print(f"=== Top 5 Similar Tickets ===")
print(f"Query: '{query_text}'\n")

for i, idx in enumerate(top_indices, 1):
    print(f"{i}. [{tickets.iloc[idx]['category']}] {tickets.iloc[idx]['ticket_text']}")
    print(f"   Similarity: {similarities[idx]:.3f}")
    print()

---

## Part 3: Semantic Search

Search tickets using natural language queries.

In [ ]:
def search_tickets(query, top_k=5):
    """
    Search tickets using a natural language query.
    """
    # Convert query to vector (using same TF-IDF)
    query_vec = tfidf.transform([query])
    
    # Calculate similarities
    sims = cosine_similarity(query_vec, ticket_vectors)[0]
    
    # Get top results
    top_idx = sims.argsort()[::-1][:top_k]
    
    results = []
    for idx in top_idx:
        results.append({
            'text': tickets.iloc[idx]['ticket_text'],
            'category': tickets.iloc[idx]['category'],
            'similarity': sims[idx]
        })
    
    return results

In [ ]:
# Try some searches
queries = [
    "refund my money",
    "package not delivered",
    "cancel subscription"
]

for q in queries:
    print(f"\n=== Query: '{q}' ===")
    results = search_tickets(q, top_k=3)
    for i, r in enumerate(results, 1):
        print(f"{i}. [{r['category']}] {r['text']} (sim: {r['similarity']:.2f})")

---

## Part 4: Limitations of TF-IDF

TF-IDF only matches exact words. It misses semantic similarity.

In [ ]:
# Test: Does TF-IDF understand synonyms?
test_queries = [
    ("I want a refund", "Give me my money back"),
    ("shipping problem", "delivery issue"),
    ("help with login", "can't access my account")
]

print("=== TF-IDF Synonym Test ===")
print("(High similarity = understands they mean the same thing)\n")

for q1, q2 in test_queries:
    v1 = tfidf.transform([q1])
    v2 = tfidf.transform([q2])
    sim = cosine_similarity(v1, v2)[0][0]
    print(f"'{q1}' ↔ '{q2}'")
    print(f"  Similarity: {sim:.2f}")
    print(f"  {('✓ Understood' if sim > 0.3 else '✗ Missed connection')}")
    print()

### The Problem

TF-IDF doesn't understand that "refund" and "money back" mean the same thing. For that, we need **neural embeddings**.

---

## Part 5: Preview - Neural Embeddings

Modern embeddings (BERT, sentence-transformers) capture meaning.

In [ ]:
# This requires sentence-transformers (optional install)
# !pip install sentence-transformers -q

neural_preview = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  METHOD              │  HOW IT WORKS                │  BEST FOR             │
├─────────────────────────────────────────────────────────────────────────────┤
│  TF-IDF              │  Word frequency counts       │  Exact keyword match  │
│                      │  Fast, interpretable         │  Known search terms   │
├─────────────────────────────────────────────────────────────────────────────┤
│  Word2Vec/GloVe      │  Average word vectors        │  Short text           │
│                      │  Captures some meaning       │  Limited context      │
├─────────────────────────────────────────────────────────────────────────────┤
│  Sentence-BERT       │  Full sentence context       │  Semantic search      │
│                      │  Understands synonyms        │  Question answering   │
└─────────────────────────────────────────────────────────────────────────────┘

💡 In practice: Start with TF-IDF. If semantic matching matters, upgrade to
   sentence-transformers. The API is almost identical!
"""
print(neural_preview)

In [ ]:
# Optional: Try neural embeddings (uncomment if sentence-transformers installed)

# from sentence_transformers import SentenceTransformer
# 
# model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, good quality
# 
# # Same test with neural embeddings
# print("=== Neural Embedding Synonym Test ===")
# for q1, q2 in test_queries:
#     v1 = model.encode([q1])
#     v2 = model.encode([q2])
#     sim = cosine_similarity(v1, v2)[0][0]
#     print(f"'{q1}' ↔ '{q2}': {sim:.2f}")

---

## Part 6: Building a Ticket Classifier

Use embeddings to auto-categorize new tickets.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    ticket_vectors, tickets['category'], test_size=0.2, random_state=42
)

# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)

print("=== Ticket Classifier Performance ===")
print(classification_report(y_test, y_pred))

In [ ]:
# Try classifying new tickets
new_tickets = [
    "I want to cancel my account",
    "Where is my package?",
    "Please refund my credit card"
]

new_vectors = tfidf.transform(new_tickets)
predictions = clf.predict(new_vectors)

print("=== New Ticket Predictions ===")
for ticket, pred in zip(new_tickets, predictions):
    print(f"'{ticket}' → {pred}")

---

## 📝 Final Exercise: Explain It

The support team asks: "How does the auto-categorization work? Is it just keyword matching?"

Write a 4-5 sentence response explaining embeddings in non-technical terms.

In [ ]:
# Write your response:

support_response = """
YOUR RESPONSE HERE
"""

print(support_response)

---

## ✅ Module 8 Complete!

**What you learned:**
- How TF-IDF converts text to vectors
- How cosine similarity finds related content
- How to build semantic search
- Limitations of TF-IDF vs neural embeddings

In [ ]:
# Summary
print("=== Module 8 Summary ===")
print(f"\nEmbedding dimensions: {ticket_vectors.shape[1]}")
print(f"Vocabulary size: {len(tfidf.vocabulary_)}")
print(f"Classifier accuracy: {(y_pred == y_test).mean():.1%}")

**Next:** [Module 9: Understanding Transformers →](./module_09_transformers.ipynb)